Imports:

In [1]:
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install scikit-learn
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import linear_model as lm


In [ ]:
Data Retrieval

In [9]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

print("Example Dialogue:")
dialogue.sample(4)


Example Dialogue:


,text,speaker,season,episode,scene,utterance
19978,"Well, I'm so sorry. I thought I was leaving yo...",Phoebe Abbott,4,1,2,12
12076,"Joey, a woman just stuck her tounge down my th...",Chandler Bing,2,22,5,6


In [10]:
episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

print("Example Episode:")
episodes.sample(4)

Example Episode:


,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
164,7,19,The One with Ross and Monica's Cousin,Gary Halvorson,Andrew Reich & Ted Cohen,2001-04-19,16.55,8.7
182,8,13,The One Where Chandler Takes a Bath,Ben Weiss,Vanessa McCarthy,2002-01-17,29.24,8.6
